In [1]:
import datetime,time,random,os
import requests,re
from bs4 import BeautifulSoup
import pandas as pd
startPage="http://www.cpppc.org:8086/pppcentral/map/getPPPList.do" #用于寻找每个项目的PROJ_ID
baseUrl='http://www.cpppc.org:8083/efmisweb/ppp/projectLibrary/getProjInfo.do?projId=' #与PROJ_ID拼接成各个项目的链接
baseUrl2='http://www.cpppc.org:8083/efmisweb/ppp/projectLibrary/getProjInfoNational.do?projId=' #另外一个链接，提取“采购社会资本方式”数据
myParams= {'queryPage': '1','projStateType':'0'} #翻页用，也可以加入其他选项
 
myHeader={
'Accept':'application/json, text/javascript, */*; q=0.01',
'Accept-Encoding':'gzip, deflate',
'Accept-Language':'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
'Connection':'keep-alive',
'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
'Cookie':'JSESSIONID=7A442574FB143BC29453ECFA1A10CCC2; _site_id_cookie=82; clientlanguage=zh_CN; faspjsessionid=201xEsLqLL6_eaSNWmga7x05Z-tO19aSbqBEJZgBol-6t836nruw!1514000050; JSESSIONID=IF12fM4-OOkAeZCsfI-E3ErRwaXDf9AAuuANnExxdWHvMxHtrAo5!-549990464',
'Host':'www.cpppc.org:8086',
'Origin':'http://www.cpppc.org:8086',
'Referer':'http://www.cpppc.org:8086/pppcentral/map/toPPPList.do',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36',
'X-Requested-With':'XMLHttpRequest'
}
def gettotalPage():
    #得到总页数
    r=requests.get(startPage,params=myParams,headers=myHeader)
    regExp="\"totalPage\":\d*"
    regResult=re.findall(regExp,r.text)[0]
    totalPage=re.sub('\"totalPage\":',"",regResult)
    totalPage=eval(totalPage) #转化为数值
    print(totalPage)
    return totalPage
 
def gettotalCount():
    #得到项目总数
    r=requests.get(startPage,params=myParams,headers=myHeader)
    regExp="\"totalCount\":\d*"
    regResult=re.findall(regExp,r.text)[0]
    totalCount=re.sub('\"totalCount\":',"",regResult)
    totalCount=eval(totalCount) #转化为数值
    print(totalCount)
    return totalCount
 
def getPageurl(Page):
    #对每一页，得到所有项目ID、NAME，并生成相应链接
    PROJ_ID=[]
    PROJ_NAME=[]
    PROJ_URL=[]
    PROJ_URL2=[]
    myParams['queryPage']=str(Page)
    try:
        r=requests.get(startPage,params=myParams,headers=myHeader,timeout=30)
    except:
        r=requests.get(startPage,params=myParams,headers=myHeader,timeout=200)
    r.encoding = 'utf-8'
 
    regExp="PROJ_NAME.*?,"
    regResult=re.findall(regExp,r.text)
    for i in range(0,len(regResult)):
        regResult[i]=re.sub('PROJ_NAME\":\"','',regResult[i])
        regResult[i]=re.sub('\",','',regResult[i])
        PROJ_NAME.append(regResult[i].strip()) 
 
    regExp="PROJ_RID.*?,"
    regResult=re.findall(regExp,r.text)
    for i in range(0,len(regResult)):
        regResult[i]=re.sub('PROJ_RID\":\"','',regResult[i])
        regResult[i]=re.sub('\",','',regResult[i])
        PROJ_ID.append(regResult[i]) 
        PROJ_URL.append(baseUrl+regResult[i])
        PROJ_URL2.append(baseUrl2+regResult[i])
    urlList=list(zip(PROJ_ID,PROJ_NAME,PROJ_URL,PROJ_URL2))
    return urlList
 
def getData(projUrl1):
    #查找每个链接的数据
    dataDict={}
    try:
        html = requests.get(projUrl1,headers=myHeader,timeout=100)
    except:
        html = requests.get(projUrl1,headers=myHeader,timeout=2000)
    html.encoding = 'utf-8'
    bsObj=BeautifulSoup(html.text,"lxml")
    #print(bsObj)
    cases = ["所在地区", "所属行业", "项目总投资", "所处阶段", "发起时间", "回报机制", "项目示范级别/批次", "合作期限(识别)", 
             "项目运作方式(识别)", "采购社会资本方式","项目公司名称","项目公司成立时间","项目公司注册资金"]
    #Table=bsObj.find("table",id="gdrjTable")
    #print(Table.text.strip())
    for i in range(len(cases)):
        dataDict[cases[i]]=bsObj.find("td",text=cases[i]).next_sibling.next_sibling.get_text().strip()
    Table=bsObj.find("table",id="gdrjTable")
    data=Table.text.strip().split("\n")
    num=int((len(data)-5)/7)
    #print(num)
    if num==0 :
        dataDict["股东认缴"]="暂无"
    else :
        dataDict["股东认缴"]="见后续"
    for i in range(1,num+1):
        #print(i)
        name="股东名称"+str(i)
        typ="政府或社会资本"+str(i)
        money="出资额（万元）"+str(i)
        ratio="股权比例（%）"+str(i)
        dataDict[name]=data[7*i+2-1]
        dataDict[typ]=data[7*i+3-1]
        dataDict[money]=data[7*i+4-1]
        dataDict[ratio]=data[7*i+5-1]
    time.sleep(random.random()*0.5)
    return dataDict

In [3]:
starttime = datetime.datetime.now()
totalPage=gettotalPage()
totalCount=gettotalCount() #totalCount仅用于计算进度
STARTPAGES=1
ENDPAGES=401
currentPage=0
currentCount=0
finalResult=[]
for i in range(STARTPAGES,ENDPAGES):
    print('-------------------------------------------------------')
    currentPage=i
    urlList=getPageurl(i)
    for k in range(0,len(urlList)):
        dataDict1={}
        dataDict1["项目名称"]=urlList[k][1]
        dataDict2=getData(urlList[k][3])
        finalResult.append(list(dataDict1.values())+list(dataDict2.values()))
        currentCount=8*i-8+k+1
        print("进度：\t"+str(currentCount)+"/"+str(totalCount)+"行\t"+str(currentPage)+"/"+str(totalPage)+"页")
    if i%100==0 :
        ###自适应列数
        List=finalResult
        need=int((max(len(List[j]) for j in range(len(List)))-15)/4)
        #print(need)
        myHeaderName= ["项目名称","所在地区", "所属行业", "项目总投资（万元）", "所处阶段", "发起时间", "回报机制", "项目示范级别/批次", 
                       "合作期限（年）", "项目运作方式", "采购社会资本方式","项目公司名称","项目公司成立时间","项目公司注册资金","股东认缴"]
        AddName= ['公司名称',"类型","出资额（万元）","股权比例（%）"]
        for m in range(1,need+1):
            myHeaderName.append(AddName[0]+str(m))
            myHeaderName.append(AddName[1]+str(m))
            myHeaderName.append(AddName[2]+str(m))
            myHeaderName.append(AddName[3]+str(m))
        myHeaderName3=myHeaderName
        tempResultOut=pd.DataFrame(columns=myHeaderName3,data=finalResult)
        if(os.path.exists(str(i)+".xlsx")):
            os.remove(str(i)+".xlsx")
        writer=pd.ExcelWriter(str(i)+".xlsx")
        tempResultOut.to_excel(writer,sheet_name='数据',index=False)
        writer.close()
print('-------------------------------------------------------')
#计算程序运行时间
endtime = datetime.datetime.now()
def timeStr(s):
    if s<10:
        return '0'+str(s)
    else:
        return str(s)
print("程序开始运行时间："+timeStr(starttime.hour)+":"+timeStr(starttime.minute)+":"+timeStr(starttime.second))
print("程序结束运行时间："+timeStr(endtime.hour)+":"+timeStr(endtime.minute)+":"+timeStr(endtime.second))
runTime=(endtime-starttime).seconds
runTimehour=runTime//3600 #除法并向下取整，整除
runTimeminute=(runTime-runTimehour*3600)//60
runTimesecond=runTime-runTimehour*3600-runTimeminute*60
print("程序运行耗时："+str(runTimehour)+"时"+str(runTimeminute)+"分"+str(runTimesecond)+"秒")

1543
12341
-------------------------------------------------------
进度：	1/12341行	1/1543页
进度：	2/12341行	1/1543页
进度：	3/12341行	1/1543页
进度：	4/12341行	1/1543页
进度：	5/12341行	1/1543页
进度：	6/12341行	1/1543页
进度：	7/12341行	1/1543页
进度：	8/12341行	1/1543页
-------------------------------------------------------
进度：	9/12341行	2/1543页
进度：	10/12341行	2/1543页
进度：	11/12341行	2/1543页
进度：	12/12341行	2/1543页
进度：	13/12341行	2/1543页
进度：	14/12341行	2/1543页
进度：	15/12341行	2/1543页
进度：	16/12341行	2/1543页
-------------------------------------------------------
进度：	17/12341行	3/1543页
进度：	18/12341行	3/1543页
进度：	19/12341行	3/1543页
进度：	20/12341行	3/1543页
进度：	21/12341行	3/1543页
进度：	22/12341行	3/1543页
进度：	23/12341行	3/1543页
进度：	24/12341行	3/1543页
-------------------------------------------------------
进度：	25/12341行	4/1543页
进度：	26/12341行	4/1543页
进度：	27/12341行	4/1543页
进度：	28/12341行	4/1543页
进度：	29/12341行	4/1543页
进度：	30/12341行	4/1543页
进度：	31/12341行	4/1543页
进度：	32/12341行	4/1543页
-------------------------------------------------------
进度：	33/12341行	

In [4]:
###自适应列数
List=finalResult
need=int((max(len(List[i]) for i in range(len(List)))-15)/4)
#print(need)
myHeaderName= ["项目名称","所在地区", "所属行业", "项目总投资（万元）", "所处阶段", "发起时间", "回报机制", "项目示范级别/批次", 
               "合作期限（年）", "项目运作方式", "采购社会资本方式","项目公司名称","项目公司成立时间","项目公司注册资金","股东认缴"]
AddName= ['公司名称',"类型","出资额（万元）","股权比例（%）"]
for i in range(1,need+1):
    myHeaderName.append(AddName[0]+str(i))
    myHeaderName.append(AddName[1]+str(i))
    myHeaderName.append(AddName[2]+str(i))
    myHeaderName.append(AddName[3]+str(i))
myHeaderName3=myHeaderName
###开始写入总数据
fileName='PPP_Database_'+str(STARTPAGES)+"-"+str(ENDPAGES)+'_Origin.xlsx'
tempResultOut=pd.DataFrame(columns=myHeaderName3,data=finalResult)
if (os.path.exists(fileName)):
   os.remove(fileName)
writer=pd.ExcelWriter(fileName)
tempResultOut.to_excel(writer,sheet_name='数据',index=False)
writer.close()